In [ ]:
from nltk.metrics.distance import edit_distance
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
## please load forget and retain

retain['text'] = retain['question'] + " " + retain['answer']
forget['text'] = forget['question'] + " " + forget['answer']

In [ ]:
def pos_seq(text):
  return [tok.pos_ for tok in nlp(text)]


forget['pos_seq'] = forget['text'].apply(pos_seq)
retain['pos_seq'] = retain['text'].apply(pos_seq)

In [ ]:
rows = []

for i,j in product(range(len(forget)), range(len(retain))):
    seq1 = forget.at[i, 'pos_seq']
    seq2 = retain.at[j, 'pos_seq']
    ed = edit_distance(seq1, seq2)
    max_len = max(len(seq1), len(seq2)) or 1
    syn_sim = round(1 - (ed / max_len),2)

    rows.append({"id_f" : forget.at[i, 'id'],
                 "id_r" : retain.at[j, 'id'],
                 "q_f" : forget.at[i, 'question'],
                 "a_f" : forget.at[i, 'answer'],
                 "q_r" : retain.at[j, 'question'],
                 "a_r" : retain.at[j, 'answer'],
                 "syn_sim" : syn_sim
                })
result = pd.DataFrame(rows)

In [ ]:
result_sorted = (
    result
    .sort_values(by=["id_f", "syn_sim"], ascending=[True, False])
    .reset_index(drop=True)
)

In [ ]:
percentage = 0.01
total_samples = int(result["id_r"].nunique() * percentage)

num_id_f = result["id_f"].nunique()

k = total_samples // num_id_f 

top_samples = (
    result_sorted
    .groupby("id_f", group_keys=False)
    .head(k)
)